#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [2]:
import pandas as pd
import pymongo

In [3]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [11]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/Users/raphael/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [10]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [2]:
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
df_ks = df_ks.dropna(subset=['launched'])
df_ks.rename(columns={'ID':'_id'}, inplace=True)

l_d =df_ks.to_dict(orient='records')

NameError: name 'pd' is not defined

### Importer les données

In [ ]:
collection.insert_many(l_d)

## Question 1  

In [ ]:
prj = collection.find().sort([('pledged',-1)]).limit(5)
list(prj)

## Question 2

In [ ]:
num = collection.count_documents({"pledged" : {'$gte' : '$goal'}})
print("Number of projects that reached their goal is : ", num)

## Question 3

In [1]:
num = collection.aggregate([{"$group" : {"_id" : "$category", "nomber of projets" : {"$sum" : 1}}}])
list(num)

NameError: name 'collection' is not defined

## Question 4

In [ ]:
import datetime
date_iso = datetime.datetime(2017,1,1,0,0,0)
num = collection.count_documents({"$and" : [{"country" : "FR"}, {"launched" : {"$lt" : date_iso}}]})
print("Number of French projects that were instantiated before 2016 is : ", num)

## Question 5

In [ ]:
num = collection.aggregate([
    {"$match" : {"$and" : [{"country" : "US"}, {"goal" : {'$gt' : 200000}}]}}
])
list(num)

## Question 6 

In [ ]:
num = collection.count_documents({"name" : {'$regex' : 'Sport'}})
print( "number of name with 'Sport' in them is :", num)